### Importing

In [1]:
import os
import glob
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder


### Downloading and Cleaning

In [2]:
# Specify the relative path to the data directory
data_folder_path = "C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Data"
non_games_data_folder_path = "C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Non_Games_Data"

# List all files in the data folder
file_list = os.listdir(data_folder_path)
file_list_non_games = os.listdir(non_games_data_folder_path)

# Use glob to filter specific file types
csv_files = glob.glob(os.path.join(data_folder_path, "*.csv"))
csv_files_non_games = glob.glob(os.path.join(non_games_data_folder_path, "*.csv"))

In [3]:
# Read in the weekly game data and concat into one combined df
dfs = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dfs, ignore_index=True)

# Read in the supplementary data
games = pd.read_csv(csv_files_non_games[0])
nfl_colors = pd.read_csv(csv_files_non_games[1])
players = pd.read_csv(csv_files_non_games[2])
plays = pd.read_csv(csv_files_non_games[3])
tackles = pd.read_csv(csv_files_non_games[4])

In [5]:
clean_df = pd.read_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/clean_df.csv")

#Full Clean
#clean_df = pd.read_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/full_clean_df.csv")


In [ ]:
# Change the orientation of the plays
clean_df = combined_df.copy()

clean_df['clean_x'] = clean_df.apply(lambda row: 120 - row['x'] if row['playDirection'] == "left" else row['x'], axis=1)
clean_df['clean_y'] = clean_df.apply(lambda row: 160 / 3 - row['y'] if row['playDirection'] == "left" else row['y'], axis=1)
clean_df['clean_dir'] = clean_df.apply(lambda row: row['dir'] + 180 if row['playDirection'] == "left" else row['dir'], axis=1)
clean_df['clean_dir'] = clean_df['clean_dir'].apply(lambda val: val - 360 if val > 360 else val)
clean_df['clean_o'] = clean_df.apply(lambda row: row['o'] + 180 if row['playDirection'] == "left" else row['o'], axis=1)
clean_df['clean_o'] = clean_df['clean_o'].apply(lambda val: val - 360 if val > 360 else val)


# Merge nfl_colors and change the color of the football
clean_df = pd.merge(clean_df, nfl_colors, left_on="club", right_on="Code", how="left")
clean_df["primary"] = np.where(
    clean_df["primary"].isna(), "#8b4513", clean_df["primary"]
)

In [ ]:
clean_df.to_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/full_clean_df.csv", index=False)

In [ ]:
print(clean_df["event"].drop_duplicates())

### Data prep and Filtering

In [17]:
# Mutate 'df'
filtered_df = clean_df.copy()

filtered_df["is_start"] = np.where(
    filtered_df["event"].isin(
        [
            "autoevent_ballsnap",
              "ball_snap",
                "pass_arrived",
                  "pass_outcome_caught",
                  "pass_outcome_touchdown",
                    "run",
                      "snap_direct",
                      "handoff"
    ]), 1, 0
)
filtered_df["is_end"] = np.where(
    filtered_df["event"].isin(
        [
            "tackle",
            "touchdown",
            "out_of_bounds",
            "fumble",
            "lateral",
            "qb_sack",
            "autoevent_passinterrupted",
            "safety",
            "autoevent_passinterrupted"
        ]
    ),
    1,
    0,
)
# Group by and mutate 'df'
grouped = filtered_df.groupby(["gameId", "playId"])
filtered_df["any_start"] = grouped["is_start"].transform("any")
filtered_df["any_end"] = grouped["is_end"].transform("any")

# Filter and summarize 'df'
intermediate_df = filtered_df[(filtered_df["any_start"]) & (filtered_df["any_end"])]


# Define a function to calculate start_frame and end_frame
def calculate_frames(group):
    start_indices = group[group["is_start"] == 1].index
    if len(start_indices) == 0:
        return pd.Series({"start_frame": None, "end_frame": None})

    start_frame = group.loc[start_indices[0], "frameId"]

    end_indices = group[(group["is_end"] == 1) & (group["frameId"] > start_frame)].index
    if len(end_indices) == 0:
        last_row_index = group.index[-1]  # Get the index of the last row in the group
        end_frame = group.loc[last_row_index, "frameId"]
    else:
        end_frame = group.loc[end_indices[0], "frameId"]

    return pd.Series({"start_frame": start_frame, "end_frame": end_frame})


# Apply the function to each group and reset index
frames_of_interest = (
    intermediate_df.groupby(["gameId", "playId"]).apply(calculate_frames).reset_index()
)

GotTheBall = plays[["gameId", "playId", "ballCarrierId"]]
GotTheBall["GotTheBall"] = 1


filtered_df["Pass"] = np.where(
    filtered_df["event"].isin(
        [
             "pass_arrived",
                  "pass_outcome_caught",
                  "pass_outcome_touchdown"
    ]), 1, 0
)

filtered_df['PassPlay'] = filtered_df.groupby(['gameId', 'playId'])['Pass'].transform('max')

PassPlay = filtered_df[['gameId', 'playId', 'PassPlay']].drop_duplicates()


C:\Users\sethl\AppData\Local\Temp\ipykernel_21120\455596652.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GotTheBall["GotTheBall"] = 1


In [10]:
# Creating a WorkDF that is filtered down to just the frames of interest
WorkDF = pd.merge(clean_df, frames_of_interest, on=["gameId", "playId"], how="inner")
WorkDF = WorkDF[
    (WorkDF["frameId"] >= WorkDF["start_frame"])
    & (WorkDF["frameId"] <= WorkDF["end_frame"])
]


WorkDF = pd.merge(WorkDF, GotTheBall,
                   left_on = ["gameId", "playId", "nflId"],
                   right_on=["gameId", "playId", "ballCarrierId"],
                   how = "left")


WorkDF = pd.merge(WorkDF, PassPlay,
                   on = ["gameId", "playId"],
                   how = "left")

In [8]:
WorkDF.to_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/small_WorkDF.csv", index=False)


### Feature Engineering

To create a feature on stunts, I would ideally want to differentiate between t-e stunts and e-t stunts

Potentially also want to create a feature that is what gap they ended up attacking. That way we can cluster based on the ending gap of each DL player and that would create more distinct clusters for the predicted path algorithm to train on.
Create gaps based on the ranges of y coordinates between the OL. I want to make it so that if you end up on the OL at the end of the play, it takes into account what orientation the OL is in. 


In [ ]:
# Filtering the WorkDF down to just the blockers
StuntDF = pd.merge(
    WorkDF, play_block_rush, on=["gameId", "playId", "nflId"], how="inner"
)


# Making Center Adjusted Coordinates for every player
StuntDF = StuntDF.sort_values(by=["gameId", "playId", "frameId", "pff_positionLinedUp"])
first_frame = StuntDF[StuntDF["pff_positionLinedUp"] == "C"].groupby(["gameId", "playId"]).transform("first")
StuntDF["C_x"] = first_frame["clean_x"]
StuntDF["C_y"] = first_frame["clean_y"]
StuntDF["C_x"] = StuntDF.groupby(["gameId", "playId"])["C_x"].fillna(method="ffill")
StuntDF["C_y"] = StuntDF.groupby(["gameId", "playId"])["C_y"].fillna(method="ffill")
StuntDF["C_adjusted_x"] = StuntDF["clean_x"] - StuntDF["C_x"]
StuntDF["C_adjusted_y"] = StuntDF["clean_y"] - StuntDF["C_y"]


GapDF = StuntDF[
    (StuntDF["pff_role"] == "Block")
    & (
        StuntDF["pff_positionLinedUp"].isin(
            ["LT", "LG", "C", "RG", "RT", "TE-L", "TE-R"]
        )
    )
]

# Creating a wider dataset with the y positions for each OL position as a new column
StuntWider = GapDF.pivot_table(
    index=["gameId", "playId", "frameId"], columns="pff_positionLinedUp", values="C_adjusted_y"
).reset_index()

# Filling the missing data for TEs to help aid assignment later
StuntWider["TE-L"] = StuntWider["TE-L"].fillna(100)
StuntWider["TE-R"] = StuntWider["TE-R"].fillna(0)

# Making a new dataframe that is just the pass rushers
Starting_Gaps = StuntDF[(StuntDF["pff_role"] == "Rush")]

# Create Cutoff Point into the play
Time_Cutoff = 20

Starting_Gaps["Cutoff_Point"] = np.where(
    Starting_Gaps["end_frame"] < Starting_Gaps["start_frame"] + Time_Cutoff,
    Starting_Gaps["end_frame"],
    Starting_Gaps["start_frame"] + Time_Cutoff,
)

Starting_Gaps = Starting_Gaps[
    (Starting_Gaps["frameId"] >= Starting_Gaps["start_frame"])
    & (Starting_Gaps["frameId"] <= Starting_Gaps["Cutoff_Point"])
]

# Joining the OL Y positional data onto the Starting_Gaps df
Starting_Gaps = pd.merge(Starting_Gaps, StuntWider, on=["gameId", "playId", "frameId"])

# Assigning the Gaps based on y position data
conditions = [
    Starting_Gaps["C_adjusted_y"] > Starting_Gaps["TE-L"],
    (Starting_Gaps["C_adjusted_y"] < Starting_Gaps["TE-L"])
    & (Starting_Gaps["C_adjusted_y"] > Starting_Gaps["LT"]),
    (Starting_Gaps["C_adjusted_y"] < Starting_Gaps["LT"])
    & (Starting_Gaps["C_adjusted_y"] > Starting_Gaps["LG"]),
    (Starting_Gaps["C_adjusted_y"] < Starting_Gaps["LG"])
    & (Starting_Gaps["C_adjusted_y"] > Starting_Gaps["C"]),
    (Starting_Gaps["C_adjusted_y"] < Starting_Gaps["C"])
    & (Starting_Gaps["C_adjusted_y"] > Starting_Gaps["RG"]),
    (Starting_Gaps["C_adjusted_y"] < Starting_Gaps["RG"])
    & (Starting_Gaps["C_adjusted_y"] > Starting_Gaps["RT"]),
    (Starting_Gaps["C_adjusted_y"] < Starting_Gaps["RT"])
    & (Starting_Gaps["C_adjusted_y"] > Starting_Gaps["TE-R"]),
    Starting_Gaps["C_adjusted_y"] < Starting_Gaps["TE-R"],
]
choices = ["L-D", "L-C", "L-B", "L-A", "R-A", "R-B", "R-C", "R-D"]

Starting_Gaps["Gap_Assignment"] = np.select(conditions, choices, default=np.NAN)

# Filling any NAN Gaps (which occur when the DL is directly on the OL Y value) with their previous Gap Assignment
Starting_Gaps["Gap_Assignment"] = Starting_Gaps.groupby(["gameId", "playId"])[
    "Gap_Assignment"
].fillna(method="ffill")


Start_Gap = Starting_Gaps[Starting_Gaps["frameId"] == Starting_Gaps["start_frame"]].groupby(["gameId", "playId", "nflId"])[["gameId", "playId", "nflId","Gap_Assignment"]].transform('first')
End_Gap = Starting_Gaps[Starting_Gaps["frameId"] == Starting_Gaps["end_frame"]].groupby(["gameId", "playId", "nflId"])[["gameId", "playId", "nflId","Gap_Assignment"]].transform('first')

Starting_Gaps = pd.merge(Starting_Gaps, Start_Gap, on=["gameId", "playId", "nflId"])
Starting_Gaps = pd.merge(Starting_Gaps, End_Gap, on=["gameId", "playId", "nflId"])

Starting_Gaps['RushPath'] = Starting_Gaps['Gap_Assignment_y'] + '-' + Starting_Gaps['Gap_Assignment'].astype(str)



In [ ]:
#Creating a One Hot Encoding for the Rush Paths types on a given play
RushPaths = Starting_Gaps.copy()

RushPaths = RushPaths.pivot_table(
    index=["gameId", "playId"], columns = "RushPath", values="frameId"
).reset_index()

exclude_columns = ["gameId", "playId"]

RushPaths.loc[:, ~RushPaths.columns.isin(exclude_columns)] = RushPaths.loc[:, ~RushPaths.columns.isin(exclude_columns)].notna().replace(True, 1)
RushPaths = RushPaths.replace(False, 0)

In [ ]:
#Creating the Number of Rushers on a play
Rushers = StuntDF.copy()

Rushers = Rushers.groupby(['gameId', 'playId', 'nflId']) \
                  .head(1) \
                  .assign(Rushers=lambda x: x.groupby(['gameId', 'playId'])['gameId'].transform('count'))

Rushers = Rushers[["gameId", "playId", "Rushers"]].drop_duplicates()

#Creating Number of Blockers on a play
Blockers = StuntDF.copy()

Blockers = Blockers[Blockers["pff_role"] == "Block"].groupby(['gameId', 'playId', 'nflId']) \
                  .head(1) \
                  .assign(Blockers=lambda x: x.groupby(['gameId', 'playId'])['gameId'].transform('count'))

Blockers = Blockers[["gameId", "playId", "Blockers"]].drop_duplicates()



In [ ]:
#Creating a One-Hot Encoding for the positions on a given play
Positions = StuntDF.copy()

Positions = Positions.pivot_table(
    index=["gameId", "playId"], columns = "pff_positionLinedUp", values="frameId"
).reset_index()

exclude_columns = ["gameId", "playId"]

Positions.loc[:, ~Positions.columns.isin(exclude_columns)] = Positions.loc[:, ~Positions.columns.isin(exclude_columns)].notna().replace(True, 1)
Positions = Positions.replace(False, 0)

In [ ]:
#Creating a One Hot Encoding for the Block types on a given play
BlockTypes = StuntDF.copy()

BlockTypes = BlockTypes.pivot_table(
    index=["gameId", "playId"], columns = "pff_blockType", values="frameId"
).reset_index()

exclude_columns = ["gameId", "playId"]

BlockTypes.loc[:, ~BlockTypes.columns.isin(exclude_columns)] = BlockTypes.loc[:, ~BlockTypes.columns.isin(exclude_columns)].notna().replace(True, 1)
BlockTypes = BlockTypes.replace(False, 0)

In [ ]:

man_zone_encoding = pd.get_dummies(man_zone, columns=['pff_passCoverage', 'pff_passCoverageType'])

play_encoding = pd.get_dummies(play_info, columns=["offenseFormation", "personnelO", "personnelD", "defendersInBox", "dropBackType", "pff_playAction"])

Center_LOC = pff_scouting_data[["gameId", "playId", "nflId", "pff_positionLinedUp"]]

# Filtering the WorkDF down to just the blockers
C_AdjustedDF = pd.merge(
    WorkDF, Center_LOC, on=["gameId", "playId", "nflId"]
)


# Making Center Adjusted Coordinates for every player
C_AdjustedDF = C_AdjustedDF.sort_values(by=["gameId", "playId", "frameId", "pff_positionLinedUp"])
first_frame = C_AdjustedDF[C_AdjustedDF["pff_positionLinedUp"] == "C"].groupby(["gameId", "playId"]).transform("first")
C_AdjustedDF["C_x"] = first_frame["clean_x"]
C_AdjustedDF["C_y"] = first_frame["clean_y"]
C_AdjustedDF["C_x"] = C_AdjustedDF.groupby(["gameId", "playId"])["C_x"].fillna(method="ffill")
C_AdjustedDF["C_y"] = C_AdjustedDF.groupby(["gameId", "playId"])["C_y"].fillna(method="ffill")
C_AdjustedDF["C_adjusted_x"] = C_AdjustedDF["clean_x"] - C_AdjustedDF["C_x"]
C_AdjustedDF["C_adjusted_y"] = C_AdjustedDF["clean_y"] - C_AdjustedDF["C_y"]

C_AdjustedDF = C_AdjustedDF[["gameId", "playId", "nflId", "frameId", "C_adjusted_x", "C_adjusted_y"]]

In [ ]:
C_AdjustedDF.to_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/C_AdjustedDF.csv", index=False)


In [ ]:
RoutesDf = pd.merge(
    WorkDF, play_route_coverage, on=["gameId", "playId", "nflId"], how="inner"
)

ClusterDF = RoutesDf[["gameId", "playId"]].drop_duplicates()

ClusterDF = pd.merge(ClusterDF, man_zone_encoding, on=["gameId", "playId"])
ClusterDF = pd.merge(ClusterDF, play_encoding, on=["gameId", "playId"])
ClusterDF = pd.merge(ClusterDF, Rushers, on=["gameId", "playId"])
ClusterDF = pd.merge(ClusterDF, Blockers, on=["gameId", "playId"])
ClusterDF = pd.merge(ClusterDF, Positions, on=["gameId", "playId"])



In [ ]:
kmeans = KMeans(n_clusters=100)  # Replace 3 with your chosen K value
kmeans.fit(ClusterDF)


In [ ]:
ClusterDF['Cluster'] = kmeans.labels_

FinalClusterJoin = ClusterDF[["gameId", "playId", "Cluster"]]



In [ ]:
ClusterDF.to_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/ClusterDF.csv", index=False)
RoutesDf.to_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/RoutesDf.csv", index=False)

#Starting_Gaps.to_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/Starting_Gaps.csv", index=False)
StuntDF.to_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/StuntDF.csv", index=False)